In [1]:
import numpy as np
import h5py
import cv2
import glob
import matplotlib.pyplot as plt
import time
import sys
import tensorflow as tf
from random import randint


# read file for `white` picture

In [2]:
def read_while_file():
    print("reading data of white...")
    file_path='d:/Special_topic/pleurotus_eryngii/white/*'
    file_name = glob.glob(file_path)
    file_num = len(file_name)

    data = np.array(file_num*[480*[640*[3*[None]]]])

    for i in range(file_num):
        data[i] = cv2.imread(file_name[i])

    data = data.astype('uint8')

    ## plot image for white data
    # plt.figure(figsize=(10,5))
    # for i in range(file_num):
    #     plt.subplot(3,3,i+1)
    #     plt.axis("off")
    #     plt.title(file_name[i])
        # print("data[i-1].shpae: ",data[i-1].shape)
        # plt.imshow(cv2.cvtColor(data[i], cv2.COLOR_BGR2RGB))
    # plt.show()

    print("reading data of white sucessful\n")
    return data


white_data = read_while_file()


reading data of white...


reading data of white sucessful



# read file for `data` picture

In [3]:
def read_data_file():
    print("reading data....")
    file_path='d:/Special_topic/pleurotus_eryngii/1030/*'
    file_name = glob.glob(file_path)
    group_num = 5 #len(file_name)//9
    print("group_num = ", group_num)

    ## [group_num][black&LED select][NumOfPicture][picture(480*640*3)]
    data = np.array(group_num*[2*[6*[480*[640*[3*[None]]]]]])
    print("data.shpae = ",data.shape)

    for group in range(group_num):
        print("processing group {} ...".format(group))

        for black in range(3):
            data[group][0][black] = cv2.imread(file_name[group*9+black])

        for LED in range(6):
            data[group][1][LED] = cv2.imread(file_name[group*9+3+LED])

    print("\nreading data sucessful\n")
    return data,group_num

#=========================================================
data, data_Gnum = read_data_file()


reading data....
group_num =  5


data.shpae =  (5, 2, 6, 480, 640, 3)
processing group 0 ...
processing group 1 ...
processing group 2 ...
processing group 3 ...


processing group 4 ...

reading data sucessful



In [4]:
# solve div0 question
def div0( a, b ):
    """ ignore / 0, div0( [-1, 0, 1], 0 ) -> [0, 0, 0] """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
        c[ ~ np.isfinite( c )] = 0  # -inf inf NaN
    return c

def print_info(img):
    print("max =",np.amax(img))
    print("min =",np.amin(img))
    print("mean=",np.mean(img))


# calculate reflectivity

In [5]:
# [group_num][NumOfPicture(LED)][picture(480*640*3)]
reflectivity = np.array(data_Gnum*[6*[480*[640*[3*[0]]]]],dtype='float32')
print("reflectivity.shape=", reflectivity.shape)
white_data = white_data.astype('float32')
data = data.astype('float32')
print()

dark_30 = white_data[2]
dark_40 = white_data[0]
dark_100= white_data[1]
# print("dark_100.shape=",dark_100.shape)

LED1_white = white_data[3]
LED2_white = white_data[4]
LED3_white = white_data[5]
LED4_white = white_data[5]
LED5_white = white_data[7]
LED6_white = white_data[8]
# print("LED6_white.shape=",LED6_whdef array_divide(a,b):

for Gnum in range(data_Gnum):
    LED1 = data[Gnum][1][0]
    LED2 = data[Gnum][1][1]
    LED3 = data[Gnum][1][2]
    LED4 = data[Gnum][1][3]
    LED5 = data[Gnum][1][4]
    LED6 = data[Gnum][1][5]
    # print("LED6.shape=",LED6.shape)

    reflectivity[Gnum][0] = div0((LED1-dark_40) , (LED1_white-dark_40))
    reflectivity[Gnum][1] = div0((LED2-dark_30) , (LED2_white-dark_30))
    reflectivity[Gnum][2] = div0((LED3-dark_30) , (LED3_white-dark_30))
    reflectivity[Gnum][3] = div0((LED4-dark_30) , (LED4_white-dark_30))
    reflectivity[Gnum][4] = div0((LED5-dark_30) , (LED5_white-dark_30))
    reflectivity[Gnum][5] = div0((LED6-dark_100), (LED6_white-dark_100))

    # temp1 = (LED1-dark_30)
    # temp2 = LED1_white-dark_30
    # temp3 = div0(temp1,temp2)

    # temp3[temp3<0] = 0
    # temp3[temp3>1] = 1

    print("processing group {} ...".format(Gnum))

reflectivity[reflectivity<0] = 0
reflectivity[reflectivity>1] = 1
print("process sucessful\n")


reflectivity.shape= (5, 6, 480, 640, 3)



processing group 0 ...
processing group 1 ...


processing group 2 ...
processing group 3 ...
processing group 4 ...


process sucessful



In [ ]:
def set_figure_size():
    plt.figure(figsize=(20,10))

# [group_num][NumOfPicture(LED)][picture(480*640*3)]
color = ['Blue','Green','Red']
for Gnum in range(data_Gnum):
    print(" group {} ==================================".format(Gnum))
    set_figure_size()
    for i in range(1,7):
        plt.subplot(1,6,i)
        plt.axis("off")
        plt.title("LED{}, {}".format(i,'true color'))
        plt.imshow(cv2.cvtColor(reflectivity[Gnum][(i-1)//3], cv2.COLOR_BGR2RGB))
    plt.show()

    set_figure_size()
    for i in range(1,7):
        for j in range(1,4):
            plt.subplot(3,6,i+(j-1)*6)
            plt.axis("off")
            plt.title("LED{}, {}".format(i,color[j-1]))
            plt.imshow(reflectivity[Gnum][i-1][:,:,j-1], plt.cm.gray, vmax=1.0)
    plt.show()


# ===================== temp =====================

In [ ]:
print(np.amin(reflectivity[0][0][:,:,0]))
print(np.amax(reflectivity[0][0][:,:,0]))
print(np.mean(reflectivity[0][0][:,:,0]))
print()
print(np.amin(reflectivity[0][0][:,:,1]))
print(np.amax(reflectivity[0][0][:,:,1]))
print(np.mean(reflectivity[0][0][:,:,1]))
print()
print(np.amin(reflectivity[0][0][:,:,2]))
print(np.amax(reflectivity[0][0][:,:,2]))
print(np.mean(reflectivity[0][0][:,:,2]))
print()



In [ ]:
file_path = 'd:/Special_topic/Pleurotus_eryngii/1030/20191122_093419_LED1.png'
temp = cv2.imread(file_path)

plt.axis("off")
plt.imshow(cv2.cvtColor(temp,cv2.COLOR_BGR2RGB))
plt.show()

plt.figure(figsize=(20,9))
plt.subplot(1,3,1)
plt.axis("off")
plt.title('blue')
plt.imshow(cv2.cvtColor(temp[:,:,0],cv2.COLOR_BGR2RGB),plt.cm.gray, vmax=255)

plt.subplot(1,3,2)
plt.axis("off")
plt.title('green')
plt.imshow(cv2.cvtColor(temp[:,:,1],cv2.COLOR_BGR2RGB),plt.cm.gray, vmax=255)

plt.subplot(1,3,3)
plt.axis("off")
plt.title('red')
plt.imshow(cv2.cvtColor(temp[:,:,2],cv2.COLOR_BGR2RGB),plt.cm.gray, vmax=255)
plt.show()

plt.axis("off")
plt.title('blue')
plt.imshow(cv2.cvtColor(white_data[0][:,:,0],cv2.COLOR_BGR2RGB),plt.cm.gray, vmax=25)
plt.show

In [ ]:
a = np.array([0,3,2,123],dtype='float32')
b = np.array([0,4,0,23],dtype='float32')
c = np.array([3,4,5,241],dtype='float32')
d = np.array([5,0,-8.3,12.4],dtype='float32')
temp1 = np.random.randint(80,size=(2,3,4))*np.random.random() - np.random.randint(100,size=(2,3,4))
temp2 = np.random.randint(90,size=(2,3,4))*np.random.random() - np.random.randint(100,size=(2,3,4))
print(type(temp1[0][0][0]))
print("*"*50)
print(temp1)
print("*"*50)
print(temp2)
print("*"*50)
temp3 = div0(temp1,temp2)
print(temp3)

print_info(temp3)





In [ ]:
# plt.imshow(cv2.cvtColor(LED1_white,cv2.COLOR_BGR2RGB))
# plt.imshow(cv2.cvtColor(dark_100,cv2.COLOR_BGR2RGB))

plt.imshow(LED6_white.astype('uint8'))
plt.show()
